You now know the following

1. Generate open-loop control from a given route

2. Simulate vehicular robot motion using bicycle/ unicycle model

Imagine you want to make an utility for your co-workers to try and understand vehicle models. 
Dashboards are common way to do this.

There are several options out there : Streamlit, Voila, Observable etc

Follow this
<a href="https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e">Medium post</a> on Jupyter Dash and see how to package what you learnt today in an interactive manner

Here is a   <a href="https://stackoverflow.com/questions/53622518/launch-a-dash-app-in-a-google-colab-notebook">stackoverflow question </a> on how to run dash applications on Collab

What can you assume?
+ Fix $v,\omega$ or $v,\delta$ depending on the model (users can still pick the actual value)
+ fixed wheelbase for bicycle model

Users can choose 
+ unicycle and bicycle models
+ A pre-configured route ("S", "inverted-S", "figure-of-eight" etc)
+ 1 of 3 values for $v, \omega$ (or $\delta$) 

In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import numpy as np

In [2]:
def bicycle_model(curr_pose, v, delta, dt=1.0, l = 0.9):
    '''
    >>> bicycle_model((0.0,0.0,0.0), 1.0, 0.0)
    (1.0, 0.0, 0.0)
    >>> bicycle_model((0.0,0.0,0.0), 0.0, np.pi/4)
    (0.0, 0.0, 0.0) 
    >>> bicycle_model((0.0, 0.0, 0.0), 1.0, np.pi/4)
    (1.0, 0.0, 1.11) 
    '''
    # write code to calculate next_pose
    # refer to the kinematic equations of a bicycle model
    x, y, theta = curr_pose
    x += v*np.cos(theta)*dt
    y += v*np.sin(theta)*dt
    theta += v*np.tan(delta)*dt/l
    
    # Keep theta bounded between [-pi, pi]
    theta = np.arctan2(np.sin(theta), np.cos(theta))
    # return calculated (x, y, theta)
    return x, y, theta
def get_open_loop_bicycle_commands(route, vc_slow=0.5, vc_fast=1, deltac=np.pi/12, dt=1.0, l=0.9):
    all_delta = []
    all_v = []
    deltas = {'straight': 0, 'left': deltac, 'right': -deltac}
    for manoeuvre, command in route:
        u = command/vc_fast/dt
        dtheta = vc_slow/l*np.tan(deltac)*dt
        v = np.deg2rad(command)/dtheta
        t_cmd = u if manoeuvre == 'straight' else v
        t_cmd = np.ceil(t_cmd).astype('int')
        all_delta += [deltas[manoeuvre]]*t_cmd
        if manoeuvre == "straight":
            all_v += [vc_fast]*t_cmd
        else:
            all_v += [vc_slow]*t_cmd
    return all_v, all_delta
def unicycle_model(curr_pose, v, w, dt=1.0):
    '''
    >>> unicycle_model((0.0,0.0,0.0), 1.0, 0.0)
    (1.0, 0.0, 0.0)
    >>> unicycle_model((0.0,0.0,0.0), 0.0, 1.0)
    (0.0, 0.0, 1.0)
    >>> unicycle_model((0.0, 0.0, 0.0), 1.0, 1.0)
    (1.0, 0.0, 1.0)
    '''
    ## write code to calculate next_pose
    # refer to the kinematic equations of a unicycle model
    x, y, theta = curr_pose
    x += v*np.cos(theta)*dt
    y += v*np.sin(theta)*dt
    theta += w*dt
    
    # Keep theta bounded between [-pi, pi]
    theta = np.arctan2(np.sin(theta), np.cos(theta))

    # return calculated (x, y, theta)
    return x, y, theta
def get_open_loop_unicycle_commands(route, vc_slow=0.5,vc_fast=1, wc=np.pi/12, dt=1.0):
    all_w = []
    all_v = []
    omegas = {'straight': 0, 'left': wc, 'right': -wc}
    for manoeuvre, command in route:
        u = np.ceil(command/vc_fast).astype('int')
        v = np.ceil(np.deg2rad(command)/wc).astype('int')
        t_cmd = u if manoeuvre == 'straight' else v
        t_cmd = np.ceil(t_cmd/dt).astype('int')
        all_w += [omegas[manoeuvre]]*t_cmd
        if manoeuvre == "straight":
            all_v += [vc_fast]*t_cmd
        else:
            all_v += [vc_slow]*t_cmd
    return all_v, all_w

In [3]:
routes = {
    "s": [("left",90),("straight",5)]*3 + [("right",90),("straight",5)]*2,
    "inv-s": [("right",90),("straight",5)]*3 + [("left",90),("straight",5)]*2,
    "fig-8": [("left",90),("straight",5)]*3 + [("right",90),("straight",5)]*4 + [("left",90),("straight",5)]*2
}

In [4]:
app = JupyterDash(__name__)

marks = {}
for i in range(6): marks[i] = str(i)
    
marks_w = {}
for i in range(0,61,15): marks_w[np.deg2rad(i)] = str(i) + "°"

app.layout = html.Div([
    html.H1("Vehicle Simulation Demo"),
    html.Label(["Model",
               dcc.Dropdown(id="model-dropdown",value='unicycle',options=[
                {'label':'unicycle','value':'unicycle'},{'label':'bicycle','value':'bicycle'}
            ])]),
    html.Label([
        "Route",
        dcc.Dropdown(
            id='route-dropdown',
            value='s', options=[
                {'label':'S','value':'s'},{'label':'inverted-S','value':'inv-s'},
                {'label':'Figure of 8','value':'fig-8'},
            ])
    ]),
    html.Label(["v_slow", dcc.Slider(id="v_slow-slider",min=0.1,max=5,value=0.5,step=0.1,marks=marks)]),
    html.Label(["v_fast", dcc.Slider(id="v_fast-slider",min=0.1,max=5,value=1,step=0.1,marks=marks)]),
    html.Label(["w/delta ", dcc.Slider(id="w-slider",min=np.deg2rad(0.1),max=np.deg2rad(60),value=np.deg2rad(30),step=np.deg2rad(1),marks=marks_w)]),
    dcc.Graph(id='graph')
])
@app.callback(
    Output("graph","figure"),
    Input("v_slow-slider","value"),
    Input("v_fast-slider","value"),
    Input("w-slider","value"),
    Input("model-dropdown", "value"),
    Input("route-dropdown", "value")
)
def update_figure(v_slow, v_fast, w, model, route_index):
#     if w == 0:
#         return None
    robot_trajectory = []
    route = routes[route_index]
    if model == "unicycle":
        all_v, all_w = get_open_loop_unicycle_commands(route,vc_slow=v_slow,vc_fast=v_fast,wc=w,dt=0.01)
    else:
        all_v, all_w = get_open_loop_bicycle_commands(route,vc_slow=v_slow,vc_fast=v_fast,deltac=w,dt=0.01)
    pose = (0, 0, np.pi/2)
    for v, w in zip(all_v, all_w):
        # store new pose
        robot_trajectory.append(pose)
        #instruction to take v, w and compute new pose
        if model == "unicycle":
            pose = unicycle_model(pose, v, w, dt=0.01)
        else:
            pose = bicycle_model(pose, v, w, dt=0.01)
    robot_trajectory.append(pose)
    robot_trajectory = np.array(robot_trajectory)   
    robot_trajectory = robot_trajectory[:,:2]
    fig = px.line(x=robot_trajectory[:,0],y=robot_trajectory[:,1])
    fig.layout.yaxis.scaleanchor = 'x'
    return fig
app.run_server(mode='inline')